# <center>Algorithmes pour le traitement de séquences.<br/>Alignement optimal et logiciel d’aide à la dététection de plagiat</center>
<center>https://perso.esiee.fr/~coustyj/Sequences/IT-4301E-TDm2.html</center>

## Question 1
**Proposer un algorithme en O(|x| × |y|) pour calculer le score d’un alignement optimal entre x et y (indications : on essaiera de se ramener à un problème connu).**

Pour calculer le score d'un alignement optimal entre x et y, il nous faut dans un premier temps comprendre de quoi il s'agit. En nous référant à la **Définition 3**, on peut lire : "le score d’un alignement (x′,y′) est la grandeur d(x′,y′) égale au nombre de caractères mal appariés entre x′ et y′. Un alignement (x′,y′) de deux séquences x et y définies sur A est dit optimal (pour d) si son score est inférieur ou égal au score de tout autre alignement de x et y."  

Les termes "mal appariés" nous mettent la puce à l'oreille. En effet, dans notre cours *"Comparaison de séquences : distance d'édition"*, nous avions noté : "\[les distances d'édition\] sont définies à partir d'opérations d'édition qui permettent de transformer un mot x en un mot y". Ces opérations (substitution, suppression, insertion) s'appliquent caractère par caractère. Si on associe un coût à chacune de ces opérations (toujours 1 pour la suppression et l'insertion et 1 pour la substitution si les caractères comparés sont différents, 0 sinon), alors faire la somme des coûts des étapes d'une transformation d'un texte 1 en un texte 2 nous permet de compter les "mauvais appariement" et donc revient à calculer le score d'un alignement entre x et y. Il en découle que scorer la transformation d'un texte 1 en un texte 2 comportant les étapes les moins coûteuses, autrement dit la distance d'édition, revient à scorer l'alignement optimal entre x et y.  

Il se trouve que nous connaissons un algorithme qui nous permet de calculer cette distance d'édition : l'algorithme de Levenshtein. Ce dernier nous permet de construire une matrice de dimension (|x|+1)\*(|y|+1), qui pour chaque (i,j) associe la distance d'édition entre xi et yj. Le dernier élément de cette matrice sera donc le score d'un alignement optimal (et de la distance d'édition) entre x et y.

## Question 2

**On considère la matrice T de taille (|x|+1) × (|y|+1) telle que T[i][j] est le score d’un alignement optimal entre xi et yj, où xi et yj désignent les préfixes de x et de y de longueur i et j respectivement.**

**Étant donnée la matrice T (obtenue par exemple avec l’algorithme de la question 1) et les séquences x et y, proposez un schéma de programme retournant un alignement optimal (x′,y′) de x et y.**

**Évaluer la complexité de calcul de l’algorithme proposé; si celle-ci n’est pas linéaire, proposez un autre algorithme de complexité linéaire.**

Notre objectif est, à partir de la matrice T obtenue grâce à l'algorithme discuté dans la question 1, de construire un alignement optimal (x′,y′) de x et y.  

Notre idée est d'utiliser les informations que la matrice T porte, pour construire un alignement optimal. L'algorithme serait une fonction à laquelle nous passerions la matrice T retournée par l'algorithme de Levenshtein, les indices i et j, ainsi que les textes x et y. Cette fonction sera récursive. Expliquons son fonctionnement par disjonction de cas :  
  * soit les lettres en x[i-1] et y[j-1] sont les même, et cela veut dire que l'on a nécessairement fait une subsitution (de coût 0), auquel cas, on rappelle la fonction en décrémentant de 1 i et j.  
  * soit les lettres en x[i-1] et y[j-1] sont différentes, auquel cas, on a forcément "payé" un coût de 1 et :  
    - soit T[i-1, j-1] est strictement inférieur à T[i, j], donc deux lettres ont été substituées, pas de décalage nécessaire, on rappelle la fonction en décrémentant i et j de 1.
    - soit T[i-1, j] est strictement inférieur à T[i, j], donc une lettre a été ajoutée au texte y, un décalage est nécessaire : on insère un "_" à l'indice j de y. Enfin, on rappelle la fonction en décrémentant i de 1.
    - soit T[i, j-1] est strictement inférieur à T[i, j], donc une lettre a été supprimée du texte y, un décalage est nécessaire : on insère un "_" à l'indice i de x. Enfin, on rappelle la fonction en décrémentant j de 1.
    
On ne négligera pas les conditions d'arrêt :
  * si i est égal à 0 : on insère autant de décalage qu'il n'y a de lettres restantes dans y
  * si j est égal à 0 : on insère autant de décalage qu'il n'y a de lettres restantes dans i
  
  
Cet algorithme a une complexité linéaire. En effet, on peut remarquer que dans chacun des cas, on rappelle notre fonction en décrémentant au moins un de nos indices i et j. Ainsi, dans le pire des cas, on aura i+j appels à notre fonction. Chacune de nos actions dans cette fonction étant en temps constant, notre algorithme est bien en temps linéaire.

# Question 3

**Implémentez les algorithmes des questions 1 et 2 et testez les en affichant un alignement optimal de texte1.txt et texte2.txt**

Importons le paquet dont nous avons besoin : *numpy*

In [1]:
import numpy as np

Implémentation de la question 1

In [2]:
def levenshtein_distance(a :str, b:str) -> int:
    n,m = len(a),len(b)
    T = np.zeros((n+1,m+1))
    T[:,0] = range(n+1)
    T[0,:] = range(m+1)
    
    for i in range(n):
        for j in range(m):
            same = int(a[i]!=b[j])
            T[i+1,j+1] = min(
                        T[i, j] + same, # substitution
                        T[i, j+1] + 1, # insertion
                        T[i+1, j] + 1  # suppression
                    )
            
    return T

Implémentation de la question 2

In [3]:
def reverse_lev(T, i, j, x, y):
    
    if i==0:
        x = ['_']*j + x
        return '%s\n%s'%(''.join(x), ''.join(y))
    if j==0:
        y = ['_']*i + y
        return '%s\n%s'%(''.join(x), ''.join(y))
    
    xi, yj = x[i-1], y[j-1]

    if xi == yj:
        return reverse_lev(T, i-1, j-1, x, y)
    
    else:
        
        if T[i-1, j-1] < T[i, j]:
            return reverse_lev(T, i-1, j-1, x, y)

        elif T[i-1, j] < T[i, j]:
            y.insert(j,'_')
            return reverse_lev(T, i-1, j, x, y)
        
        else:
            x.insert(i,'_')
            return reverse_lev(T, i, j-1, x, y)

Test en affichant un alignement optimal de texte1.txt et de texte2.txt

In [4]:
# Lecture des textes
texte1 = open('texte1.txt').read()
texte2 = open('texte2.txt').read()

# Génération de la matrice T contenant les coûts d'édition pour chaque couple (xi, yj)
T = levenshtein_distance(texte1, texte2)
print("Score d’un alignement optimal entre texte1.txt et texte2.txt : ", T[-1, -1], "\n")

# Affichage d'un alignement optimal
print("Affichage d'un alignement optimal :\n\n", reverse_lev(T, T.shape[0]-1, T.shape[1]-1, list(texte1), list(texte2)))

Score d’un alignement optimal entre texte1.txt et texte2.txt :  577.0 



NameError: name 't1' is not defined

# Question 4

**La méthode proposée aux questions précédentes fonctionne correctement pour aligner un texte comprenant une seule ligne ou un seul paragraphe. On se propose désormais d’aligner et mettre en correspondance les différentes lignes/paragraphes d’un texte. Il s’agit donc de chercher à apparier les lignes (et non pas les caractères) d’un premier texte avec celles d’un second. Proposez une méthode, un algorithme de résolution et une implémentation pour cette nouvelle fonctionnalité (indications : on cherchera à se ramener un problème connu, dont on possède déjà une implémentation :)).**

**Vous pourrez tester votre méthode sur les fichiers t1.txt et t2.txt. Vous devriez obtenir un résultat qui ressemble fortement à celui de la Figure 2**

In [ ]:
class ParagraphLevenshtein:
    
    def __init__(self, text1, text2):
        print('Initialisation...')
        self.text1 = text1
        self.text2 = text2
        self.x = text1.split('\n')
        self.y = text2.split('\n')
        self.n = len(self.x)
        self.m = len(self.y)
        self.distances = np.zeros((self.n,self.m)) 
        self.T = np.zeros((self.n+1,self.m+1))
        
        print('Calcul des distances...')
        self.fill_distances()
        
        print('Calcul des alignements optimaux...')
        self.lev_by_par()
        
    def fill_distances(self):
        """Sauve toutes les distances d'edition entre xi et yj"""
        di = self.distances
        
        for i in range(self.n):
            xi = self.x[i]
            for j in range(self.m):
                yj = self.y[j]
                di[i,j] = levenshtein_distance(xi,yj)[-1,-1]
        
    def lev_by_par(self) -> "2d array of paragraph alignments":
        T, x, y, n, m = self.T, self.x, self.y, self.n, self.m # pour la lisibilité
        
        for i in range(n):
            T[i+1,0] = T[i,0]+len(x[i])
        for j in range(m):
            T[0,j+1] = T[0,j]+len(y[j])

        for i in range(n):
            xi = x[i]
            for j in range(m):
                yj = y[j]
                T[i+1,j+1] = min(
                                T[i,j] + self.distances[i,j],
                                T[i+1,j] + len(yj),
                                T[i,j+1] + len(xi)
                            )
    def stretched_x_and_y(self, i, j):
        if i==0 : return ['\n%s'%self.y[k] for k in range(j)]
        if j==0 : return ['%s\n'%self.x[k] for k in range(i)]
        T = self.T
        xi = self.x[i-1]
        yj = self.y[j-1]
        
        if T[i,j] == T[i-1,j-1] + self.distances[i-1,j-1]:
            lev = levenshtein_distance(xi, yj)
            rev = reverse_lev(lev, lev.shape[0]-1, lev.shape[1]-1, list(xi), list(yj))
            return self.stretched_x_and_y(i-1,j-1) + [rev]
            
        elif T[i,j] == T[i, j-1] + len(yj):
            rev = '_'*len(yj) + '\n' + yj
            return self.stretched_x_and_y(i,j-1) + [rev]
            
        else:
            rev = xi + '\n' + '_'*len(xi)
            return self.stretched_x_and_y(i-1,j) + [rev]

In [ ]:
def side_by_side_print(array, char_by_line):
    
    wrapper = textwrap.TextWrapper(width=char_by_line)
    
    
    for sub in array:
        
        left, right = sub.split('\n')
        lw = wrapper.wrap(left)
        rw = wrapper.wrap(right)
        lw_bigger = len(lw) > len(rw)
        
        print('\n\t', '*'*char_by_line,'\t\t', '*'*char_by_line, '\n')
        
        for i in range(min(len(lw), len(rw))):
            if lw[i] and rw[i]:
                print('|\t{:<{x}}\t|\t{:<{x}}\t|'.format(lw[i], rw[i], x=char_by_line))

        if lw_bigger:
            for i in range(len(rw), len(lw)):
                print('|\t{:<{x}}\t|\t{:<{x}}\t|'.format(lw[i], '', x=char_by_line))
                
        else:
            for i in range(len(lw), len(rw)):
                print('|\t{:<{x}}\t|\t{:<{x}}\t|'.format('', rw[i], x=char_by_line))

    print('\n\t', '*'*char_by_line,'\t\t', '*'*char_by_line, '\n')


In [ ]:
import textwrap 

t1 = open('t1.txt').read()
t2 = open('t2.txt').read()

a = ParagraphLevenshtein(t1,t2)
res = a.stretched_x_and_y(a.n,a.m)

print('Distances entre les textes : ', a.T[-1,-1])
print('Longueur de t1 : ', len(t1))
print('Longueur de t2 : ', len(t2))
print('Score de similarité en % : ', 100*(len(t1)+len(t2)-a.T[-1,-1])/(len(t1)+len(t2)))

side_by_side_print(res, 45)